In [ ]:
%load_ext aiida
%aiida

In [ ]:
# General imports.
import numpy as np
import re
from datetime import datetime
import ipywidgets as ipw
from IPython.display import clear_output

from aiida.engine import ExitCode, ToContext, WorkChain, calcfunction

In [ ]:
# validate e-mail

def validemail(s):
    pat = "^[a-zA-Z0-9-_]+\.[a-zA-Z0-9-_]+@[a-zA-Z0-9]+\.[a-z]{1,3}$"
    if re.match(pat,s):
        return True
    return False

In [ ]:
style = {'description_width': '160px'}
layout = {'width': '40%'}
output = ipw.Output()
add_person = ipw.Button(description='Add person')

name = ipw.Text(description='Name and Surname',style=style, layout=layout)
email = ipw.Text(description='e-mail',style=style, layout=layout)
startdate = ipw.DatePicker(description='date',value=datetime.now(),style=style, layout=layout)
display(ipw.VBox([startdate,name,email, add_person,output]))



@calcfunction
def new_person(name,email,startdate):
    return Dict({'name':name.value,'email':email.value,'started':startdate.value})
    
#def guess_email(b):
#    if name.value != '':
#        splitname=name.value.split
#        email.value=splitname[0]+'.'+splitname[1]+'@empa.ch'




def on_add_person_clicked(b):
    with output:
        clear_output()
        if validemail(email.value) and startdate.value is not None and name.value != '':
            #new_person(Str(description.value),Str(startdate.value.ctime()), Str(email.value))
            new = new_person(Str(name.value),Str(email.value),Str(startdate.value.ctime()))
            new.label = 'nanotech@coffee'
            print("Added",new['name'],'starting from: ',new['started'],'pk:',new.pk )
            
#name.observe(guess_email)
add_person.on_click(on_add_person_clicked)

# Actions

In [ ]:
@calcfunction
def new_event(person,datei,datef,event,description,amount):
    person.set_extra(datetime.now().ctime(),{'event':event.value,'dates':[datei.value,datef.value],'description':description.value,'amount':amount.value})

In [ ]:
event = ipw.Dropdown(options=['Bought coffee','Add cash','Request cash','absent'],
                     description='Possible actions',style=style, layout=layout)
date = ipw.DatePicker(description='date',value=datetime.now(),style=style, layout=layout)
datef = ipw.DatePicker(description='date2',value=datetime.now(),style=style, layout=layout)
description = ipw.Text(description='Description',style=style, layout=layout)
amount = ipw.FloatText(description='Amount',style=style, layout=layout)
who = ipw.Dropdown(options=[],description='Person',style=style, layout=layout)

apply = ipw.Button(description='Apply')

available_cash = 10

def date_is_valid(date,person):
    if 'left' in person.extras:
        dateleft = datetime.strptime(person['left'], '%a %b %d %H:%M:%S %Y')
    else:
        dateleft = datetime.strptime('Sun Feb 12 15:48:05 2040', '%a %b %d %H:%M:%S %Y')
    return datetime.strptime(person['started'], '%a %b %d %H:%M:%S %Y') < datetime.strptime(date.ctime(), '%a %b %d %H:%M:%S %Y') < dateleft

def on_apply_clicked(b):
    with output:
        clear_output()
        person = load_node(who.value)
        if date_is_valid(date.value,person) and date_is_valid(datef.value,person):
            if event.value == 'Request cash' and amount.value > available_cash:
                print("Not enough cash available")
            elif event.value == 'absent':
                amount.value = np.busday_count(date.value.strftime("%Y-%m-%d"),datef.value.strftime("%Y-%m-%d"))
                new_event(person,Str(date.value.ctime()),Str(datef.value.ctime()),Str(event.value),Str(description.value),Float(amount.value))
                print("Added",event.value,'on',date.value.ctime(),'for',person['name'],'amount:',amount.value)
        else:
            print("Dates not valid for ", person['name'])
apply.on_click(on_apply_clicked)

In [ ]:
display(ipw.VBox([ipw.HBox([date,datef]),event,description,amount,who,apply,output]))

In [ ]:
#filters["ctime"] = {"and": [{"<=": end_date}, {">": start_date}]}

qb = QueryBuilder()
qb.append(Node, filters={
    'label': {'in': ['nanotech@coffee']}
})
##qb.order_by({WorkChainNode:{'ctime':'asc'}})

In [ ]:
options=[]
for node_tuple in qb.iterall():
    node = node_tuple[0]
    options.append((node['name'],node.pk))
who.options = options